<a href="https://colab.research.google.com/github/pallavrouth/AI-Bootcamp/blob/main/python%20scripts/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
cigarettes = pd.read_csv('https://raw.githubusercontent.com/pallavrouth/AI-Bootcamp/main/Data/cigarettes.csv')
cigarettes.head()

In [ ]:
cigarettes = pd.read_csv('https://raw.githubusercontent.com/pallavrouth/AI-Bootcamp/main/Data/cigarettes.csv',
                         index_col = 0)
cigarettes.head()

**Seleting columns**

I like to use loc with double quotes.


In [ ]:
# select state, population column
# : does not require brackets
cigarettes.loc[:,['state','population']].head()
# other methods
cigarettes.iloc[:,[0]] # can't use names
cigarettes.state # not very flexible
cigarettes['state'] # does not return data frame
cigarettes.filter(regex = 'state') # regex spec

**Indexing or slicing**

I like to use loc here

In [ ]:
# no need for double brackets if selecting a range
cigarettes.iloc[1:10,:]
# use double brackets if selecting specific rows
cigarettes.iloc[[1,2,4,6,10],:] 

If you want to select and slice at the same time, I like to use iloc with double quotes. A tip here is to use the `get_loc` function to find the integer location of the column

In [ ]:
cigarettes.columns.get_loc('population')

**Filtering**

I like to use loc here. Feels intuitive. Syntax is `data.loc[(condition),[cols]]`

condition is specified as `data['col'] condition`

In [ ]:
# filter by state : AZ and AL
cigarettes.loc[(cigarettes['state'] == "AL"),['state','population','income']]
# using is in for multiple states
cigarettes.loc[(cigarettes['state'].isin(['AL','AZ'])),['state','population','income']]
# filter rows where tax is less than 25
cigarettes.loc[(cigarettes['tax'] < 25),['state','tax']]
# filter rows where tax is less than 30 and price greater than 100
cigarettes.loc[(cigarettes['tax'] > 30) &
               (cigarettes['price'] > 200),['state','tax','price']]

In [ ]:
# you can chain locs together for multiple filters
cigarettes.loc[(cigarettes['tax'] > 30),['state','tax','price']].loc[(cigarettes['price'] > 200),:]

In [ ]:
# more to less condensed way
cigarettes[cigarettes.tax > 70]
cigarettes[cigarettes['tax'] > 70]
cigarettes.loc[cigarettes.tax > 70,:] # added flexibility of selecting specific columns
cigarettes.loc[cigarettes.tax > 70,['state']]

**Mutate**

In [ ]:
# find ratio of price to tax
cigarettes.assign(ratio_tax = lambda x: x['price']/x['tax']).head()

In [ ]:
# also define a function like this
def find_ratio(x,y) : return x/y
cigarettes.assign(ratio_tax = find_ratio(cigarettes['price'],cigarettes['tax'])).head()

Possible to do this. But the changes are permanent. 

In [ ]:
#cigarettes['ratio_tax'] = cigarettes['price']/cigarettes['tax']
#cigarettes

**Arrange**

In [ ]:
# sort by prices
cigarettes.sort_values('price',ascending = False).head()
cigarettes.sort_values(['year','price']).head()

**Misc column operations**

1. Rename
2. Drop columns
3. Reshaping data - melting (gather) and pivoting (spread)

In [ ]:
cigarettes.rename(columns = {'year':'yr'}).head()
cigarettes.drop(columns = ['year']).head()
cigarettes.pivot(index = 'state', columns = 'year', values = 'income').head()

**Group by operations**

1. Group by and summarise

Note : loc or iloc does not work with group by. So, you need to use simple `[[]]` to get a column

In [ ]:
# straight forward simple group by operations 
cigarettes['state'].value_counts()
cigarettes['state'].describe()

In [ ]:
# group by state and find mean price
cigarettes.groupby('state')['price'].mean()
# group by state and year and find mean price - use double brackets
cigarettes.groupby(['state','year'])[['price']].mean().head()

Similarly we have `count()`,`sum()` and many others - 
https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.apply.html

In [ ]:
# multiple group by operations on same column
cigarettes.groupby('state')[['price']].agg(['mean','min'])
# multiple group by operations on multiple columns
cigarettes.groupby('state').agg({'price':'mean','tax':'min'})
# the above can be extended to any number of operations

2. Group by and arrange

In [ ]:
cigarettes.groupby('state').apply(lambda x : x['price'].sort_values())
# this method returns the original data frame unlike above
cigarettes.groupby('state').apply(pd.DataFrame.sort_values,'price')
cigarettes.groupby('state').apply(pd.DataFrame.sort_values,['price','taxs'],ascending = True)

3. Group by and mutate

In [ ]:
# group by year and find the proportion of packs sold by every state
# lambda x where x is the group
cigarettes.groupby('year').apply(lambda x : x['packs']/x['packs'].sum()) 

The above method is the fastest. Cigarettes group by creates a data frame. Then I use apply function to a specific column in the data frame. Note that assign does not work on grouped data. 

To keep the whole data intact, you have to do this - use transform/apply on grouped data to keep the original data structure intact. And then you have to pass this to the assign function

In [ ]:
# using sum or apply returns the summarised data which cannot be utilized within assign later on
cigarettes.groupby('year')['packs'].sum()
# use transform or apply for that
cigarettes.groupby('year').apply(lambda x : x['packs']/x['packs'].sum())
cigarettes.groupby('year')['packs'].transform(lambda x: x.sum())
cigarettes.groupby('year')[['packs']].transform(lambda x : x.sum())

# use this inside assign
cigarettes.assign(sum_packs = cigarettes.groupby('year')[['packs']].transform(lambda x : x.sum()))

# complete function
cigarettes.assign(sum_packs = cigarettes.groupby('year')[['packs']].transform(lambda x : x.sum()),
                  prop_packs = lambda x : x['packs']/x['sum_packs'])

**Apply function**

1. In normal mutate situations: Using the `[[]]` creates a data frame object which helps in preserving the data frame object

In [ ]:
cigarettes[['price']].apply(lambda x : x ** 2).head()

You can use assign or apply inside assign. I think the latter is more intuitive and easier to break down

In [ ]:
cigarettes.assign(sq_price = lambda x : x.price ** 2)
cigarettes.assign(sq_price = cigarettes[['price']].apply(lambda x : x ** 2))

In [ ]:
# more examples
cigarettes[['price','tax']].apply(lambda x : x.price + x.tax, axis = 1)
cigarettes.assign(price_tax = cigarettes[['price','tax']].apply(lambda x : x.price + x.tax, axis = 1))

In [ ]:
# understand difference between apply and transform here
cigarettes.groupby('year')[['packs']].apply(sum)
cigarettes.groupby('year')[['packs']].transform(sum)